"""
Code adapted from: https://shephexd.github.io/development/2017/02/19/pycuda.html

If using google colab:
* Click on Runtime (excecution) and select Change runtime type (modifier le type d'excecution).
  Then select GPU in Hardware Acceleration (accélérateur matériel)
* Start your session by installing pycuda with the command:

  -> !pip install pycuda
"""

"""
Multiples two square matrices together using multiple blocks and shared memory.
Each thread block is assigned a "tile" of the resulting matrix and is responsible
for generating the elements in that tile.  Each thread in a block computes one element
of the tile.
"""

In [2]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 5.6MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621085 sha256=689f0e911a05b8f0e3b7db1efaa752b24241e796d77ad194182142851dffa850
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=8d9dda432e4ba69040e565200b6b75fc945c45a78bea1c238a15c8a0398ea8b9
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [3]:
import numpy as np
from numpy import linalg as la
from pycuda import driver, compiler, gpuarray, tools
import time

In [4]:
# -- initialize the device
import pycuda.autoinit

In [65]:
# define the (square) matrix size
#  note that we'll only use *one* block of threads here
#  as a consequence this number (squared) can't exceed max_threads
# -> use MyDevice.get_attributes() to get this information
MATRIX_SIZE = 1024

def matmul(a_gpu,b_gpu,MATRIX_SIZE=MATRIX_SIZE):
    kernel_code_template = """
    __global__ void MatrixMulKernel(float *A, float *B, float *C)
    {

      const uint wA = %(MATRIX_SIZE)s;
      const uint wB = %(MATRIX_SIZE)s;

      // Block index
      const uint bx = blockIdx.x;
      const uint by = blockIdx.y;

      // Thread index
      const uint tx = threadIdx.x;
      const uint ty = threadIdx.y;

      // Index of the first sub-matrix of A processed by the block
      const uint aBegin = wA * %(BLOCK_SIZE)s * by;
      // Index of the last sub-matrix of A processed by the block
      const uint aEnd = aBegin + wA - 1;
      // Step size used to iterate through the sub-matrices of A
      const uint aStep = %(BLOCK_SIZE)s;

      // Index of the first sub-matrix of B processed by the block
      const uint bBegin = %(BLOCK_SIZE)s * bx;
      // Step size used to iterate through the sub-matrices of B
      const uint bStep = %(BLOCK_SIZE)s * wB;

      // The element of the block sub-matrix that is computed
      // by the thread
      float Csub = 0;
      // Loop over all the sub-matrices of A and B required to
      // compute the block sub-matrix
      for (int a = aBegin, b = bBegin;
           a <= aEnd;
           a += aStep, b += bStep)
        {
          // Shared memory for the sub-matrix of A
          __shared__ float As[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];
          // Shared memory for the sub-matrix of B
          __shared__ float Bs[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];

          // Load the matrices from global memory to shared memory
          // each thread loads one element of each matrix
          As[ty][tx] = A[a + wA * ty + tx];
          Bs[ty][tx] = B[b + wB * ty + tx];
          // Synchronize to make sure the matrices are loaded
          __syncthreads();

          // Multiply the two matrices together;
          // each thread computes one element
          // of the block sub-matrix
          for (int k = 0; k < %(BLOCK_SIZE)s; ++k)
            Csub += As[ty][k] * Bs[k][tx];

          // Synchronize to make sure that the preceding
          // computation is done before loading two new
          // sub-matrices of A and B in the next iteration
          __syncthreads();
        }

      // Write the block sub-matrix to global memory;
      // each thread writes one element
      const uint c = wB * %(BLOCK_SIZE)s * by + %(BLOCK_SIZE)s * bx;
      C[c + wB * ty + tx] = Csub;
    }
    """
        # define size of blocks and tiles sub-matrix
    # (we assume that the block size is same as tile size)
    TILE_SIZE = 32
    BLOCK_SIZE = TILE_SIZE

    # get the kernel code from the template
    # by specifying the constants MATRIX_SIZE and BLOCK_SIZE
    kernel_code = kernel_code_template % {
        'MATRIX_SIZE': MATRIX_SIZE,
        'BLOCK_SIZE': BLOCK_SIZE,
        }

    # compile the kernel code
    mod = compiler.SourceModule(kernel_code)

    # create empty gpu array for the result (C = A * B)
    c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

    # get the kernel function from the compiled module
    matrixmul = mod.get_function("MatrixMulKernel")

    # call the kernel on the card
    time_start=time.time()
    matrixmul(
        # inputs
        a_gpu, b_gpu,
        # output
        c_gpu,
        # grid of multiple blocks
        grid = (MATRIX_SIZE // TILE_SIZE, MATRIX_SIZE // TILE_SIZE),
        # block of multiple threads
        block = (TILE_SIZE, TILE_SIZE, 1),
        )
    time_end=time.time()
    print('enlapsed time (GPU):',time_end-time_start,' seconds')

    return c_gpu, time_end-time_start

In [61]:
# create two random square matrices
a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)

In [62]:
# compute reference on the CPU to verify GPU computation
time_start=time.time()
c_cpu = np.dot(a_cpu, b_cpu)
time_end=time.time()
timeCPU = time_end-time_start
print('enlapsed time (CPU):',timeCPU,' seconds')

enlapsed time (CPU): 0.0404362678527832  seconds


In [63]:
# transfer host (CPU) memory to device (GPU) memory
a_gpu = gpuarray.to_gpu(a_cpu)
b_gpu = gpuarray.to_gpu(b_cpu)

# calculate the multiplication
c_gpu, timeGPU = matmul(a_gpu,b_gpu)

enlapsed time (GPU): 0.0001423358917236328  seconds


In [64]:
# print the results
def display(verbose=False):

  print("Taille matrices : ", MATRIX_SIZE)
  #print Matrices
  if verbose == True:
    print("-" * 80)
    print("Matrix A (GPU):")
    print(a_gpu.get())

    print("-" * 80)
    print("Matrix B (GPU):")
    print(b_gpu.get())

    print("-" * 80)
    print("Matrix C (GPU):")
    print(c_gpu.get())

    print("-" * 80)

  #print difference
  print("CPU-GPU difference:")
  norm = np.linalg.norm(c_cpu - c_gpu.get())
  if norm != 0:
    print(c_cpu - c_gpu.get())
  print("Norme of the difference : ", norm)

  #print difference time
  print()
  print("Rapport de temps CPU/GPU : ", round(timeCPU/timeGPU,3))

display()

Taille matrices :  1024
CPU-GPU difference:
[[ 0.0000000e+00 -1.1444092e-05 -1.0848045e-05 ...  6.6757202e-06
  -7.6293945e-06  1.9848347e-05]
 [ 1.9073486e-06 -9.5367432e-06 -7.6293945e-06 ... -1.6689301e-06
  -3.0517578e-05 -1.5258789e-05]
 [-2.4795532e-05 -3.0517578e-05 -7.6293945e-06 ... -1.9073486e-06
   0.0000000e+00 -8.5830688e-06]
 ...
 [ 6.1035156e-05 -6.6757202e-06 -5.7220459e-06 ...  1.1444092e-05
   1.1444092e-05  1.9073486e-06]
 [ 4.1961670e-05 -5.7220459e-06  9.2983246e-06 ...  1.5258789e-05
   0.0000000e+00 -1.5258789e-05]
 [-1.9073486e-05  1.2874603e-05  3.8146973e-06 ... -9.5367432e-07
  -1.5258789e-05 -6.1988831e-06]]
Norme of the difference :  0.018887706

Rapport de temps CPU/GPU :  284.09


#QUESTION 1: Comprenez bien chaque partie du code

Done

#QUESTION 2: Comparez les gain de temps de la multiplication en CPU a celle en GPU avec ceux de l'exercice precedent. Comment expliquez vous que la methode GPU soit largement competitive maintenant ?

Cette fois-ci, la taille de la matrice est plus importante, ce qui rend le gain de temps de la parallélisation plus net. De plus, on a séparé les Threads en blocs, ce qui a donc amélioré le rendement du processus.

#QUESTION 3: Jouez avec la taille de la matrice et celle des tuiles (tile). Est-ce que le gain de temps en utilisant le GPU depend fortement de la taille de la matrice ? Est-ce que le choix de la taille des tuiles a une grande influence sur les gains de temps de calculs ?

La taille de la matrice fait varier le gain de temps en utilisant le GPU.
La taille des tuiles n'a pas une grande influence sur les gains de temps de calculs.